## JSON 만들기

In [30]:
#-*-coding: utf-8
json_hierarchy = [
    {
        "div" : "서울특별시장"
    },
    {
        "div" : "대변인",
        "child" : "언론담당관"
    },
    {
        "div": "감사위원회",
        "child": "조사담당관/안전담당관/감사담당관"
    },
    {
        "div": "시민감사옴부즈만위원회",
    },
    {
        "div": "서울혁신기획관",
        "child": "인권담당관/지역공동체담당관/사회혁신담당관/청년정책담당관/갈등조정담당관/민관협력담당관"
    },
    {
        "div": "시민소통기획관",
        "child": "시민소통담당관/시민봉사담당관/도시브랜드담당관/신속행정담당관/뉴미디어담당관"
    },
    {
        "div": "행정1부시장",
    },
    {
        "div": "여성가족정책실",
        "child": "가족담당관/외국인다문화담당관/여성정책담당관/보육담당관"
    },
    {
        "div": "민생사법경찰단",
    },
    {
        "div": "비상기획관",
        "child": "민방위담당관"
    },
    {
        "div": "정보기획관",
        "child": "통계데이터담당관/정보기획담당관/정보통신보안담당관/공간정보담당관/정보시스템담당관"
    },
    {
        "div": "일자리노동정책관",
        "child": "사회적경제담당관/노동정책담당관/일자리정책담당관"
    },
    {
        "div": "행정2부시장",
    },
    {
        "div": "도시공간개선단",
    },
    {
        "div": "기술심사담당관", 
    },
    {
        "div": "정무부시장",
    }
    ]

## JSON을 이용해 url 만들기

In [31]:
from urllib import parse

# URL 추출
urls = [];
tail = '&dept%5B%5D='
def extract_value(data, origin_url):
  if (len(data)) > 2:
    for single_div in data :
      url = str(origin_url)
      if isinstance(single_div, dict):
        try:
          if isinstance(single_div['child'], str):
            #요기 최종
            ## pares 안해주면 한글 url이 깨진다요
            url += parse.quote(single_div['div'])
            url += tail
            for div in single_div['child'].split('/'):
              url_final = url + parse.quote(div)
              urls.append(url_final)
#               print(url_final)
              urls.append(url_final)
          else:
            url += parse.quote(single_div['div'])
            url += tail
            extract_value(single_div['child'], url)
        except KeyError:
#           url += parse.quote(single_div.get('div'))
#           urls.append(url)
          print('keyerr')
      else:
        print("no dict : ")
  else:
    url = str(origin_url)
    child_list = data.get('child')
    for child in child_list.split('/'):
      url_final = url + parse.quote(data.get('div')) + tail + parse.quote(child)
      urls.append(url_final)
#       print(url_final)
      
extract_value(json_hierarchy, "")
urls[:5]

keyerr
keyerr
keyerr
keyerr
keyerr
keyerr
keyerr
keyerr


['%EB%8C%80%EB%B3%80%EC%9D%B8&dept%5B%5D=%EC%96%B8%EB%A1%A0%EB%8B%B4%EB%8B%B9%EA%B4%80',
 '%EB%8C%80%EB%B3%80%EC%9D%B8&dept%5B%5D=%EC%96%B8%EB%A1%A0%EB%8B%B4%EB%8B%B9%EA%B4%80',
 '%EA%B0%90%EC%82%AC%EC%9C%84%EC%9B%90%ED%9A%8C&dept%5B%5D=%EC%A1%B0%EC%82%AC%EB%8B%B4%EB%8B%B9%EA%B4%80',
 '%EA%B0%90%EC%82%AC%EC%9C%84%EC%9B%90%ED%9A%8C&dept%5B%5D=%EC%A1%B0%EC%82%AC%EB%8B%B4%EB%8B%B9%EA%B4%80',
 '%EA%B0%90%EC%82%AC%EC%9C%84%EC%9B%90%ED%9A%8C&dept%5B%5D=%EC%95%88%EC%A0%84%EB%8B%B4%EB%8B%B9%EA%B4%80']

## 크롤링

In [32]:
# 필요한 라이브러리 가져오기
from urllib import request
!pip install html5lib
import html5lib
!pip install lxml
import pandas as pd
import numpy as np

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [50]:
import re
# 2017년 1년치 데이터 크롤링
df = pd.DataFrame()
base_url_head = "https://opengov.seoul.go.kr/expense?dept%5B%5D=%EC%84%9C%EC%9A%B8%EC%8B%9C%EB%B3%B8%EC%B2%AD&dept%5B%5D="
base_url_tail = "&year=2017&month="
column_div = "1소속부서"

# url돌면서 url 만들기
for div in urls:
  div_url = (base_url_head + div + base_url_tail)
  # url 첫부분 가져와서 df에 1소속부서 column에 넣어주기
  unquote_div = parse.unquote(div)
  div_name = re.search('(.*)&', unquote_div).group(1)
  for month in range(1,13): 
    url_month = div_url + str(month)
    try:
      df_month = pd.read_html(url_month)[1]
      df_month[column_div] = div_name
      df = df.append(df_month)
    except IndexError:
      print('err')

err https://opengov.seoul.go.kr/expense?dept%5B%5D=%EC%84%9C%EC%9A%B8%EC%8B%9C%EB%B3%B8%EC%B2%AD&dept%5B%5D=%EA%B0%90%EC%82%AC%EC%9C%84%EC%9B%90%ED%9A%8C&dept%5B%5D=%EC%A1%B0%EC%82%AC%EB%8B%B4%EB%8B%B9%EA%B4%80&year=2017&month=5
err https://opengov.seoul.go.kr/expense?dept%5B%5D=%EC%84%9C%EC%9A%B8%EC%8B%9C%EB%B3%B8%EC%B2%AD&dept%5B%5D=%EA%B0%90%EC%82%AC%EC%9C%84%EC%9B%90%ED%9A%8C&dept%5B%5D=%EC%A1%B0%EC%82%AC%EB%8B%B4%EB%8B%B9%EA%B4%80&year=2017&month=5
err https://opengov.seoul.go.kr/expense?dept%5B%5D=%EC%84%9C%EC%9A%B8%EC%8B%9C%EB%B3%B8%EC%B2%AD&dept%5B%5D=%EA%B0%90%EC%82%AC%EC%9C%84%EC%9B%90%ED%9A%8C&dept%5B%5D=%EC%95%88%EC%A0%84%EB%8B%B4%EB%8B%B9%EA%B4%80&year=2017&month=1
err https://opengov.seoul.go.kr/expense?dept%5B%5D=%EC%84%9C%EC%9A%B8%EC%8B%9C%EB%B3%B8%EC%B2%AD&dept%5B%5D=%EA%B0%90%EC%82%AC%EC%9C%84%EC%9B%90%ED%9A%8C&dept%5B%5D=%EC%95%88%EC%A0%84%EB%8B%B4%EB%8B%B9%EA%B4%80&year=2017&month=2
err https://opengov.seoul.go.kr/expense?dept%5B%5D=%EC%84%9C%EC%9A%B8%EC%8B%9C%EB%B3

크롤링 하는데 엄청 오래걸리네...

## 데이터 확인하기

In [51]:
df.describe()

,연번,집행금액
count,18206.000000,1.820600e+04
mean,29.982094,1.079157e+05
std,32.038078,1.296718e+05
min,1.000000,2.700000e+03
25%,8.000000,4.232500e+04
50%,17.000000,7.500000e+04
75%,39.000000,1.310000e+05
max,150.000000,3.595300e+06


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18206 entries, 0 to 32
Data columns (total 9 columns):
연번       18206 non-null int64
부서명      18204 non-null object
집행일시     18206 non-null object
집행장소     18206 non-null object
집행목적     18182 non-null object
대상인원     18200 non-null object
결제방법     18130 non-null object
집행금액     18206 non-null int64
1소속부서    18206 non-null object
dtypes: int64(2), object(7)
memory usage: 1.4+ MB


In [46]:
df.shape

(4664, 9)

In [53]:
df.head(2)

,연번,부서명,집행일시,집행장소,집행목적,대상인원,결제방법,집행금액,1소속부서
0,1,대변인 언론담당관,2017-01-01 13:09,육대장 (중구 세종대로),주말 당번 노고 격려 간담회,행정팀장 등 3명,카드,24000,대변인
1,2,대변인 언론담당관,2017-01-01 17:38,마마스 (중구 무교로),주말 당번 노고 격려 간담회,행정팀장 등 3명,카드,23600,대변인


In [54]:
df.tail(2)

,연번,부서명,집행일시,집행장소,집행목적,대상인원,결제방법,집행금액,1소속부서
31,32,일자리노동정책관 일자리정책담당관,2017-12-27 12:38,곰국시집(중구 무교동 12-1),자치구 부서장회의 안건 검토 간담회,일자리노동정책관 외 5명,카드,164000,일자리노동정책관
32,33,일자리노동정책관 일자리정책담당관,2017-12-28 18:41,닥터로빈 시청점(중구 세종대로 20길 23),키오스크 시연 대비 간담회,청년일자리팀장 외 9명,카드,200000,일자리노동정책관


## Tidy 1차 : column 분리하기

In [67]:
# 집행장소와 주소를 분리하자
df_copy = df.copy()
df_copy['집행지주소'] = df_copy['집행장소'].str.extract(r'\((.*)\)')
df_copy['집행지주소'] = df_copy['집행지주소'].replace(' ', '')
df_copy['집행지명'] = df_copy['집행장소'].str.extract(r'(.*)\(')
df_copy['집행지명'] = df_copy['집행지명'].replace(' ', '')
del df_copy['집행장소']
df_copy.head(3)

,연번,부서명,집행일시,집행목적,대상인원,결제방법,집행금액,1소속부서,집행지주소,집행지명
0,1,대변인 언론담당관,2017-01-01 13:09,주말 당번 노고 격려 간담회,행정팀장 등 3명,카드,24000,대변인,중구 세종대로,육대장
1,2,대변인 언론담당관,2017-01-01 17:38,주말 당번 노고 격려 간담회,행정팀장 등 3명,카드,23600,대변인,중구 무교로,마마스
2,3,대변인 언론담당관,2017-01-02 19:27,당번 매뉴얼 작성 업무협의 간담회,행정팀장 등 2명,카드,17000,대변인,중구 을지로,명동칼국수


In [68]:
# 필요없는 Column 없애기
del df_copy['연번']
del df_copy['부서명']
df_copy.head(3)

,집행일시,집행목적,대상인원,결제방법,집행금액,1소속부서,집행지주소,집행지명
0,2017-01-01 13:09,주말 당번 노고 격려 간담회,행정팀장 등 3명,카드,24000,대변인,중구 세종대로,육대장
1,2017-01-01 17:38,주말 당번 노고 격려 간담회,행정팀장 등 3명,카드,23600,대변인,중구 무교로,마마스
2,2017-01-02 19:27,당번 매뉴얼 작성 업무협의 간담회,행정팀장 등 2명,카드,17000,대변인,중구 을지로,명동칼국수


In [69]:
# 집행일시를 년/월/일/시간으로 분리하자
df_copy['year'] = df_copy['집행일시'].str.extract(r'(20..)')
df_copy['month'] = df_copy['집행일시'].str.extract(r'-(\d\d)-')
df_copy['day'] = df_copy['집행일시'].str.extract(r'-(\d\d\s)')
df_copy['time'] = df_copy['집행일시'].str.extract(r'(\d?\d:\d?\d)')
df_copy.head(3)

,집행일시,집행목적,대상인원,결제방법,집행금액,1소속부서,집행지주소,집행지명,year,month,day,time
0,2017-01-01 13:09,주말 당번 노고 격려 간담회,행정팀장 등 3명,카드,24000,대변인,중구 세종대로,육대장,2017,01,01,13:09
1,2017-01-01 17:38,주말 당번 노고 격려 간담회,행정팀장 등 3명,카드,23600,대변인,중구 무교로,마마스,2017,01,01,17:38
2,2017-01-02 19:27,당번 매뉴얼 작성 업무협의 간담회,행정팀장 등 2명,카드,17000,대변인,중구 을지로,명동칼국수,2017,01,02,19:27


In [70]:
del df_copy['집행일시']
df_copy.head(3)

,집행목적,대상인원,결제방법,집행금액,1소속부서,집행지주소,집행지명,year,month,day,time
0,주말 당번 노고 격려 간담회,행정팀장 등 3명,카드,24000,대변인,중구 세종대로,육대장,2017,01,01,13:09
1,주말 당번 노고 격려 간담회,행정팀장 등 3명,카드,23600,대변인,중구 무교로,마마스,2017,01,01,17:38
2,당번 매뉴얼 작성 업무협의 간담회,행정팀장 등 2명,카드,17000,대변인,중구 을지로,명동칼국수,2017,01,02,19:27


In [71]:
# '대변인' 테이블만 보기
df_copy[df_copy['1소속부서'] == '대변인'].head(3)

,집행목적,대상인원,결제방법,집행금액,1소속부서,집행지주소,집행지명,year,month,day,time
0,주말 당번 노고 격려 간담회,행정팀장 등 3명,카드,24000,대변인,중구 세종대로,육대장,2017,01,01,13:09
1,주말 당번 노고 격려 간담회,행정팀장 등 3명,카드,23600,대변인,중구 무교로,마마스,2017,01,01,17:38
2,당번 매뉴얼 작성 업무협의 간담회,행정팀장 등 2명,카드,17000,대변인,중구 을지로,명동칼국수,2017,01,02,19:27


## 파일로 저장하기

In [76]:
# 크롤링하는데 오래 걸리니 데이터를 파일로 저장해두자
from IPython.display import FileLink, FileLinks

df_copy.to_csv('C:\workspace\djangoPrj\popular_restaurants_from_officials/seoul_dining_crawl.csv', index=False)
df_copy.to_excel('C:\workspace\djangoPrj\popular_restaurants_from_officials/seoul_dining_crawl.xlsx', index=False)

FileLinks('C:\workspace\djangoPrj\popular_restaurants_from_officials')

C:\workspace\djangoPrj\popular_restaurants_from_officials/
  data_crawling.ipynb
  README.md
  seoul_dining.xlsx
  seoul_dining_crawl.csv
  seoul_dining_crawl.xlsx
  서울시업무추진비데이터_2.ipynb
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git/
  COMMIT_EDITMSG
  config
  description
  FETCH_HEAD
  HEAD
  index
  ORIG_HEAD
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\hooks/
  applypatch-msg.sample
  commit-msg.sample
  fsmonitor-watchman.sample
  post-update.sample
  pre-applypatch.sample
  pre-commit.sample
  pre-push.sample
  pre-rebase.sample
  pre-receive.sample
  prepare-commit-msg.sample
  update.sample
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\info/
  exclude
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\logs/
  HEAD
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\logs\refs\heads/
  master
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\logs\refs\remotes\origin/
  master
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\01/
  a688294847d7286d9359d325a03443552661b4
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\02/
  f258f9289a5382860b61dc0970673704cd36c6
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\03/
  357ce4f5833006114f2e790a7b89dbf69d4f90
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\04/
  73643c419af10e77427f5841c8d773d513e538
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\05/
  f3a6e5eb189936b17e93f28e86dc781e800f04
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\06/
  cfe42c68a7e5255cd1171e9ec4e1897a8678b9
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\09/
  7a86c699a42ed9e5b071b3b98cdee0abb9eafc
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\0a/
  096502da9b95076187df64b7f06376429c4a17
  a4b144b7d9d0cc947e4efe298b1fade7eb3c14
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\0b/
  cad6b506bd1e18b0144883d7f383e469077ef0
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\0c/
  bda561b9598626ea42cd5453b3ef69e69f3c0c
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\0d/
  070d2af1cb403b96adec78f49f9e14d05914c6
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\0f/
  435abcc307c02f8583856c9d37400a6d9965f5
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\10/
  a9e05be7c8f4b9b63fa1b8a4492cf9dd6e67ea
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\15/
  60c7b4a97586facb0a365183d872a2cc2db400
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\17/
  690a3478dae152fb4599c17ec97adbef5d7d69
  d1ad67cdcca17f6ddcdbb4edf062a9f2b49b60
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\18/
  4df53b1b936a65f7fb225ac51761227471a311
  4df9240ce76e1508086ac5bef6600c81809643
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\1a/
  14959bcadac9d036d4d3296177437a94a884f4
  d2537d0b45e87ca9216d6377b6a78f28b04b20
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\1b/
  771fb608f2eea1516ad2ea3674d5d0f715ea17
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\1c/
  6b99fc0946c3f41df99174e3621eb88d3c23e7
  a64ae5b08ed18efda27c9a58a8496d31afac2a
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\1d/
  e874cf891a8fcf97097b49fb78762a5332eb87
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\1f/
  a47a7f82926403b2912a43ae6ef21318c3119f
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\21/
  efa265b59b10b9e9ad8cc3fe796973126d44f6
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\22/
  709beb443064c0fc11fbdd9e6590e5e751ffcf
  8854f3b00be502dbb2deed17020bbfe915556d
C:\workspace\djangoPrj\popular_restaurants_from_officials\.git\objects\24/
  55e486009ed704e215faf3453d1

## Top20 집계

In [80]:
df_copy['cnt'] = 1
df_copy.groupby('집행지명').sum().sort_values(by='cnt', ascending=False)[:20]

,집행금액,cnt
집행지명,,
참숯골,26890560,242
우도일식,24248000,202
참숯골,15484400,164
구이구이,13180000,146
동해일식,17008000,136
바닷가작은부엌,17676000,116
스타벅스,2592260,116
파리바게뜨,6503540,106
우도일식,13382000,104


- 참숯골, 우도일식, 구이구이, 곰국시집이 여러개다. 왜지...?
- '파리바게[트뜨]', '스타벅스'는 빼야겠다
- '시청'이 들어가는 곳도 빼야겠어